<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Time Series to Image Classification with tsai-rs

This notebook demonstrates converting time series to images for classification using **tsai-rs**.

## Introduction

Sometimes it may be useful to transform a univariate or multivariate time series into an image so that any of the techniques available for images can be used.

tsai-rs provides efficient Rust implementations of several time series to image transforms:

* **GASF** (Gramian Angular Summation Field): Creates images based on angular differences
* **GADF** (Gramian Angular Difference Field): Creates images based on angular sums
* **Recurrence Plot**: Shows recurring patterns in the time series

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
import matplotlib.pyplot as plt

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Available TS to Image Transforms

The following time series to image transforms are available in tsai-rs:

| Transform | Description | Input Range |
|-----------|-------------|-------------|
| `compute_gasf` | Gramian Angular Summation Field | [-1, 1] |
| `compute_gadf` | Gramian Angular Difference Field | [-1, 1] |
| `compute_recurrence_plot` | Recurrence Plot | Any |

## Load Sample Data

In [ ]:
# Load multivariate dataset
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape} (samples, variables, length)")
print(f"Classes: {np.unique(y_train)}")

In [ ]:
# Standardize and prepare data
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)

print(f"Standardized data range: [{X_train_std.min():.2f}, {X_train_std.max():.2f}]")

## Visualizing All Transforms

In [ ]:
# Get a single univariate time series
sample_idx = 0
var_idx = 0
ts = X_train_std[sample_idx, var_idx, :].astype(np.float32)

# Compute all transforms
gasf = tsai_rs.compute_gasf(ts)
gadf = tsai_rs.compute_gadf(ts)
rp = tsai_rs.compute_recurrence_plot(ts, threshold=0.1)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].plot(ts)
axes[0, 0].set_title('Original Time Series')
axes[0, 0].set_xlabel('Time')

im1 = axes[0, 1].imshow(gasf, cmap='viridis', aspect='auto')
axes[0, 1].set_title(f'GASF ({gasf.shape[0]}x{gasf.shape[1]})')
plt.colorbar(im1, ax=axes[0, 1])

im2 = axes[1, 0].imshow(gadf, cmap='plasma', aspect='auto')
axes[1, 0].set_title(f'GADF ({gadf.shape[0]}x{gadf.shape[1]})')
plt.colorbar(im2, ax=axes[1, 0])

im3 = axes[1, 1].imshow(rp, cmap='binary', aspect='auto')
axes[1, 1].set_title(f'Recurrence Plot ({rp.shape[0]}x{rp.shape[1]})')
plt.colorbar(im3, ax=axes[1, 1])

plt.tight_layout()
plt.show()

## GASF (Gramian Angular Summation Field)

GASF encodes time series into images by representing the temporal correlation between different time points as angular sums.

In [ ]:
# GASF for different samples
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    ts = X_train_std[i, 0, :].astype(np.float32)
    gasf = tsai_rs.compute_gasf(ts)
    
    axes[0, i].plot(ts)
    axes[0, i].set_title(f'Sample {i} - Class {y_train[i]}')
    
    axes[1, i].imshow(gasf, cmap='viridis', aspect='auto')
    axes[1, i].set_title(f'GASF')

axes[0, 0].set_ylabel('Time Series')
axes[1, 0].set_ylabel('GASF Image')
plt.tight_layout()
plt.show()

## GADF (Gramian Angular Difference Field)

GADF is similar to GASF but uses angular differences instead of sums, capturing different characteristics of the time series.

In [ ]:
# GADF for different samples
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    ts = X_train_std[i, 0, :].astype(np.float32)
    gadf = tsai_rs.compute_gadf(ts)
    
    axes[0, i].plot(ts)
    axes[0, i].set_title(f'Sample {i} - Class {y_train[i]}')
    
    axes[1, i].imshow(gadf, cmap='plasma', aspect='auto')
    axes[1, i].set_title(f'GADF')

axes[0, 0].set_ylabel('Time Series')
axes[1, 0].set_ylabel('GADF Image')
plt.tight_layout()
plt.show()

## Recurrence Plot

Recurrence plots visualize the times at which a phase space trajectory visits roughly the same area in the phase space.

In [ ]:
# Recurrence plots with different thresholds
thresholds = [0.05, 0.1, 0.2, 0.5]
ts = X_train_std[0, 0, :].astype(np.float32)

fig, axes = plt.subplots(1, len(thresholds), figsize=(16, 4))

for i, thresh in enumerate(thresholds):
    rp = tsai_rs.compute_recurrence_plot(ts, threshold=thresh)
    axes[i].imshow(rp, cmap='binary', aspect='auto')
    axes[i].set_title(f'Threshold = {thresh}')

plt.suptitle('Recurrence Plots with Different Thresholds')
plt.tight_layout()
plt.show()

## Comparing Classes with Image Transforms

In [ ]:
# Get unique classes
unique_classes = np.unique(y_train)
n_classes = len(unique_classes)

print(f"Classes: {unique_classes}")

# Create GASF images for one sample from each class
fig, axes = plt.subplots(3, n_classes, figsize=(18, 12))

for i, cls in enumerate(unique_classes):
    # Get first sample of this class
    idx = np.where(y_train == cls)[0][0]
    ts = X_train_std[idx, 0, :].astype(np.float32)
    
    # Time series
    axes[0, i].plot(ts)
    axes[0, i].set_title(f'Class {cls}')
    
    # GASF
    gasf = tsai_rs.compute_gasf(ts)
    axes[1, i].imshow(gasf, cmap='viridis', aspect='auto')
    axes[1, i].set_title('GASF')
    
    # Recurrence Plot
    rp = tsai_rs.compute_recurrence_plot(ts, threshold=0.1)
    axes[2, i].imshow(rp, cmap='binary', aspect='auto')
    axes[2, i].set_title('Recurrence Plot')

axes[0, 0].set_ylabel('Time Series')
axes[1, 0].set_ylabel('GASF')
axes[2, 0].set_ylabel('Recurrence Plot')

plt.tight_layout()
plt.show()

## Multivariate Time Series to Images

For multivariate time series, you can create one image per variable or stack them as channels.

In [ ]:
# Create images for multiple variables
sample = X_train_std[0]  # Shape: (n_vars, seq_len)
n_vars_to_show = min(6, sample.shape[0])

fig, axes = plt.subplots(3, n_vars_to_show, figsize=(18, 10))

for i in range(n_vars_to_show):
    ts = sample[i, :].astype(np.float32)
    
    # Time series
    axes[0, i].plot(ts)
    axes[0, i].set_title(f'Var {i}')
    
    # GASF
    gasf = tsai_rs.compute_gasf(ts)
    axes[1, i].imshow(gasf, cmap='viridis', aspect='auto')
    
    # GADF
    gadf = tsai_rs.compute_gadf(ts)
    axes[2, i].imshow(gadf, cmap='plasma', aspect='auto')

axes[0, 0].set_ylabel('Time Series')
axes[1, 0].set_ylabel('GASF')
axes[2, 0].set_ylabel('GADF')

plt.suptitle('Multivariate Time Series to Images')
plt.tight_layout()
plt.show()

## Creating Image Datasets

In [ ]:
def ts_to_image_dataset(X, transform='gasf', threshold=0.1):
    """
    Convert time series dataset to images.
    
    Args:
        X: Input data (samples, vars, length)
        transform: 'gasf', 'gadf', or 'rp'
        threshold: Threshold for recurrence plot
    
    Returns:
        images: Array of images (samples, vars, height, width)
    """
    n_samples, n_vars, seq_len = X.shape
    
    # First, compute one image to get the output shape
    sample_ts = X[0, 0, :].astype(np.float32)
    if transform == 'gasf':
        sample_img = tsai_rs.compute_gasf(sample_ts)
    elif transform == 'gadf':
        sample_img = tsai_rs.compute_gadf(sample_ts)
    elif transform == 'rp':
        sample_img = tsai_rs.compute_recurrence_plot(sample_ts, threshold=threshold)
    else:
        raise ValueError(f"Unknown transform: {transform}")
    
    img_h, img_w = sample_img.shape
    images = np.zeros((n_samples, n_vars, img_h, img_w), dtype=np.float32)
    
    for i in range(n_samples):
        for j in range(n_vars):
            ts = X[i, j, :].astype(np.float32)
            if transform == 'gasf':
                images[i, j] = tsai_rs.compute_gasf(ts)
            elif transform == 'gadf':
                images[i, j] = tsai_rs.compute_gadf(ts)
            elif transform == 'rp':
                images[i, j] = tsai_rs.compute_recurrence_plot(ts, threshold=threshold)
    
    return images

# Create a small subset for demonstration
X_subset = X_train_std[:10]

# Convert to images
gasf_images = ts_to_image_dataset(X_subset, transform='gasf')
gadf_images = ts_to_image_dataset(X_subset, transform='gadf')
rp_images = ts_to_image_dataset(X_subset, transform='rp', threshold=0.1)

print(f"Original shape: {X_subset.shape}")
print(f"GASF images shape: {gasf_images.shape}")
print(f"GADF images shape: {gadf_images.shape}")
print(f"RP images shape: {rp_images.shape}")

## Stacking Images as Channels

In [ ]:
def create_multichannel_image(ts, include_gasf=True, include_gadf=True, include_rp=True, threshold=0.1):
    """
    Create a multi-channel image from different transforms.
    
    Args:
        ts: 1D time series (float32)
        include_*: Which transforms to include
        threshold: Threshold for recurrence plot
    
    Returns:
        Multi-channel image
    """
    channels = []
    
    if include_gasf:
        channels.append(tsai_rs.compute_gasf(ts))
    if include_gadf:
        channels.append(tsai_rs.compute_gadf(ts))
    if include_rp:
        channels.append(tsai_rs.compute_recurrence_plot(ts, threshold=threshold))
    
    return np.stack(channels, axis=0)

# Create a 3-channel image (like RGB)
ts = X_train_std[0, 0, :].astype(np.float32)
multichannel = create_multichannel_image(ts)

print(f"Multi-channel image shape: {multichannel.shape}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(multichannel[0], cmap='viridis')
axes[0].set_title('Channel 0: GASF')

axes[1].imshow(multichannel[1], cmap='plasma')
axes[1].set_title('Channel 1: GADF')

axes[2].imshow(multichannel[2], cmap='binary')
axes[2].set_title('Channel 2: RP')

plt.tight_layout()
plt.show()

## Data Augmentation with Images

In [ ]:
# Apply augmentation before converting to images
ts_original = X_train_std[0, 0, :].astype(np.float32)

# Create augmented versions
ts_augmented = [ts_original.copy()]
labels = ['Original']

for std in [0.05, 0.1]:
    aug = tsai_rs.add_gaussian_noise(X_train_std[:1].astype(np.float32), std=std, seed=42)
    ts_augmented.append(aug[0, 0, :])
    labels.append(f'Noise std={std}')

for scale in [(0.8, 0.9), (1.1, 1.2)]:
    aug = tsai_rs.mag_scale(X_train_std[:1].astype(np.float32), scale_range=scale, seed=42)
    ts_augmented.append(aug[0, 0, :])
    labels.append(f'Scale={scale}')

# Visualize augmented images
fig, axes = plt.subplots(len(ts_augmented), 3, figsize=(12, 3 * len(ts_augmented)))

for i, (ts, label) in enumerate(zip(ts_augmented, labels)):
    axes[i, 0].plot(ts)
    axes[i, 0].set_title(f'{label} - Time Series')
    
    gasf = tsai_rs.compute_gasf(ts)
    axes[i, 1].imshow(gasf, cmap='viridis', aspect='auto')
    axes[i, 1].set_title('GASF')
    
    rp = tsai_rs.compute_recurrence_plot(ts, threshold=0.1)
    axes[i, 2].imshow(rp, cmap='binary', aspect='auto')
    axes[i, 2].set_title('Recurrence Plot')

plt.tight_layout()
plt.show()

## Testing on Different Datasets

In [ ]:
datasets = ['ECG200', 'GunPoint', 'Wafer']

fig, axes = plt.subplots(len(datasets), 4, figsize=(16, 4 * len(datasets)))

for i, dsid in enumerate(datasets):
    try:
        X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)
        X_std = tsai_rs.ts_standardize(X_train[:1].astype(np.float32), by_sample=True)
        ts = X_std[0, 0, :]
        
        axes[i, 0].plot(ts)
        axes[i, 0].set_title(f'{dsid} - Time Series')
        
        gasf = tsai_rs.compute_gasf(ts)
        axes[i, 1].imshow(gasf, cmap='viridis', aspect='auto')
        axes[i, 1].set_title('GASF')
        
        gadf = tsai_rs.compute_gadf(ts)
        axes[i, 2].imshow(gadf, cmap='plasma', aspect='auto')
        axes[i, 2].set_title('GADF')
        
        rp = tsai_rs.compute_recurrence_plot(ts, threshold=0.1)
        axes[i, 3].imshow(rp, cmap='binary', aspect='auto')
        axes[i, 3].set_title('Recurrence Plot')
        
    except Exception as e:
        print(f"Error with {dsid}: {e}")

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated time series to image conversion with tsai-rs:

### Available Transforms
- `compute_gasf`: Gramian Angular Summation Field
- `compute_gadf`: Gramian Angular Difference Field
- `compute_recurrence_plot`: Recurrence Plot (with configurable threshold)

### Key Points
1. **Standardize first**: Use `ts_standardize` before image transforms
2. **Single channel per variable**: Each variable produces one image channel
3. **Stack transforms**: Combine GASF, GADF, RP as RGB-like channels
4. **Augment before transform**: Apply noise/scaling before image conversion

### Use Cases
- Leverage pretrained vision models (ResNet, VGG, etc.)
- Visual inspection of time series patterns
- Multi-modal learning (combine with raw TS features)

In [ ]:
# Quick reference
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

# Standardize
X_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)

# Get single time series
ts = X_std[0, 0, :]

# Apply transforms
gasf = tsai_rs.compute_gasf(ts)
gadf = tsai_rs.compute_gadf(ts)
rp = tsai_rs.compute_recurrence_plot(ts, threshold=0.1)

print(f"Time series shape: {ts.shape}")
print(f"GASF shape: {gasf.shape}")
print(f"GADF shape: {gadf.shape}")
print(f"Recurrence plot shape: {rp.shape}")